05 Alternative regression model using MCMC + maximum likelihood 

> Implemented using PyMC

In [1]:
#|default_exp core.05_alt_regression_model

In [2]:
#|hide
#import nbdev; nbdev.nbdev_export()

In [3]:
#|hide 
from nbdev.showdoc import show_doc

In [4]:
#|export
import dementia_inequalities as proj 
from dementia_inequalities import const, log, utils, tools 
import adu_proj.utils as adutils 

In [40]:
#|export 
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import KFold, cross_val_score

# pymc
import pymc as pm
from pymc import Normal, HalfNormal

#pyro
import os
import torch
import pyro
from torch import nn 
from pyro.nn import PyroModule


---

In [6]:
#|export 
df_dem_plus = pd.read_csv(const.output_path+'/df_dem_plus.csv')

In [7]:
df_dem_plus.head()

,area_code,area_name,pop,DEM_afflicted,HYP_afflicted,DM_afflicted,STIA_afflicted,ALevel_plus,Female_all,Male_all,...,Male_over_65,over_65,white_pc,DEM_afflicted_pc,ALevel_plus_pc,over_65_pc,female_pc,HYP_afflicted_pc,DM_afflicted_pc,STIA_afflicted_pc
0,E07000091,New Forest,151481,1858.546949,28892.139020,9338.695410,4466.648271,72749.0,91513.3596,84729.6073,...,23644.9796,51920.6826,0.932,0.012269,0.480252,0.342754,0.604124,0.190731,0.061649,0.029487
1,E09000027,Richmond upon Thames,156795,1016.262785,16263.714161,5723.913130,2019.443594,112919.0,101226.6730,93918.3682,...,14170.0187,31637.7606,0.630,0.006481,0.720170,0.201778,0.645599,0.103726,0.036506,0.012880
2,E07000116,Tunbridge Wells,95022,876.255260,12908.434533,4606.674451,1889.024705,52395.0,59540.1412,56126.1458,...,10171.4566,22570.0090,0.842,0.009222,0.551399,0.237524,0.626593,0.135847,0.048480,0.019880
3,E07000121,Lancaster,123214,1228.376774,16806.421220,6784.714317,2647.583108,65724.0,72615.6480,69496.5141,...,13527.6316,29465.7648,0.878,0.009969,0.533413,0.239143,0.589346,0.136400,0.055064,0.021488
4,E07000042,Mid Devon,67987,547.793487,10854.465333,4281.878984,1530.207441,33190.0,42541.3752,40639.6312,...,9396.6343,20046.6806,0.938,0.008057,0.488182,0.294860,0.625728,0.159655,0.062981,0.022507


In [26]:
# X and y data 

# covariates 
covar = df_dem_plus[['over_65_pc', 'female_pc', 'ALevel_plus_pc']].values.reshape(-1,3)
#covar = df_dem_plus[['over_65_pc', 'female_pc', 'ALevel_plus_pc', 'white_pc', 'HYP_afflicted_pc', 'DM_afflicted_pc', 'STIA_afflicted_pc']].values.reshape(-1,7)
covar = covar.T

# Outcome data 
Y_data = df_dem_plus['DEM_afflicted_pc'].values.reshape(-1,1)

In [27]:
covar[0].shape

(306,)

In [33]:
basic_model = pm.Model()

with basic_model:
    # priors 
    alpha = pm.Normal("alpha", mu=0, sigma=1)
    beta = pm.Normal("beta", mu=0, sigma=1, shape=3)
    sigma = pm.HalfNormal("sigma", sigma=1)

    # expected value of line
    line = alpha + beta[0]*covar[0]+beta[1]*covar[1]+beta[2]*covar[2]

    # likelihood 
    #y_obs = line + pm.HalfNormal("y_obs", sigma=sigma, observed=Y_data)
    Y_obs = pm.Normal("Y_obs", mu=line, sigma=sigma, observed=Y_data)

In [35]:
# not able to call the model as it leads to lots of errors - so think I probably can't use pymc :( 

# with basic_model:
#     # draw 1000 posterior samples
#     idata = pm.sample()

In [42]:
# for CI testing
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.6')
pyro.set_rng_seed(1)

In [43]:
# try pyro instead

# X and y data 

# covariates 
data = torch.tensor(df_dem_plus[['over_65_pc', 'female_pc', 'ALevel_plus_pc', 'DEM_afflicted_pc']].values, dtype=torch.float)

x_data, y_data = data[:,:-1], data[:,-1]

linear_reg_model = PyroModule[nn.Linear](3,1)

# Define loss and optimize
loss_fn = torch.nn.MSELoss(reduction='sum')
optim = torch.optim.Adam(linear_reg_model.parameters(), lr=0.05)
num_iterations = 1500 if not smoke_test else 2

def train():
    # run the model forward on the data
    y_pred = linear_reg_model(x_data).squeeze(-1)
    # calculate the mse loss
    loss = loss_fn(y_pred, y_data)
    # initialize gradients to zero
    optim.zero_grad()
    # backpropagate
    loss.backward()
    # take a gradient step
    optim.step()
    return loss

for j in range(num_iterations):
    loss = train()
    if (j + 1) % 50 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))


# Inspect learned parameters
print("Learned parameters:")
for name, param in linear_reg_model.named_parameters():
    print(name, param.data.numpy())

[iteration 0050] loss: 0.0446
[iteration 0100] loss: 0.0221
[iteration 0150] loss: 0.0176
[iteration 0200] loss: 0.0135
[iteration 0250] loss: 0.0099
[iteration 0300] loss: 0.0070
[iteration 0350] loss: 0.0048
[iteration 0400] loss: 0.0033
[iteration 0450] loss: 0.0022
[iteration 0500] loss: 0.0014
[iteration 0550] loss: 0.0010
[iteration 0600] loss: 0.0007
[iteration 0650] loss: 0.0005
[iteration 0700] loss: 0.0004
[iteration 0750] loss: 0.0003
[iteration 0800] loss: 0.0003
[iteration 0850] loss: 0.0003
[iteration 0900] loss: 0.0003
[iteration 0950] loss: 0.0003
[iteration 1000] loss: 0.0003
[iteration 1050] loss: 0.0003
[iteration 1100] loss: 0.0003
[iteration 1150] loss: 0.0003
[iteration 1200] loss: 0.0003
[iteration 1250] loss: 0.0003
[iteration 1300] loss: 0.0003
[iteration 1350] loss: 0.0003
[iteration 1400] loss: 0.0003
[iteration 1450] loss: 0.0003
[iteration 1500] loss: 0.0003
Learned parameters:
weight [[ 0.0289444  -0.0044787  -0.00163908]]
bias [0.00429869]
